In [ ]:
%pip install langchain_openai langchain langchain_experimental ollama

In [71]:
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import tool

# Define your tools!

For this we create a Langchain tool for retrieving weather information and use the `convert_to_openai_function` to create the function definition that we will feed into the model

In [67]:
class SearchInput(BaseModel):
    location: str = Field(description="The city and state, " "e.g. San Francisco, CA")

@tool("get_weather_tool", args_schema=SearchInput)
def get_current_weather(location: str) -> str:
    """Get the current weather in a given location"""
    return {"location": location, "fahrenheit": 73.4}

tools = [get_current_weather]
functions = [convert_to_openai_function(t) for t in tools]

functions

[{'name': 'get_weather_tool',
  'description': 'get_weather_tool(location: str) -> str - Get the current weather in a given location',
  'parameters': {'type': 'object',
   'properties': {'location': {'description': 'The city and state, e.g. San Francisco, CA',
     'type': 'string'}},
   'required': ['location']}}]

# Invoke the Model

Now we can prompt the model and pass the functions 🥳

In [69]:
import ollama
import json

SYSTEM_PROMPT = f"""
You are a weather assistant with access to these functions -
{json.dumps(functions, indent=4)}
"""

print (SYSTEM_PROMPT)


You are a weather assistant with access to these functions -
[
    {
        "name": "get_weather_tool",
        "description": "get_weather_tool(location: str) -> str - Get the current weather in a given location",
        "parameters": {
            "type": "object",
            "properties": {
                "location": {
                    "description": "The city and state, e.g. San Francisco, CA",
                    "type": "string"
                }
            },
            "required": [
                "location"
            ]
        }
    }
]



In [122]:
import json
import re
from typing import Optional, Dict

def parse_function_call(input_str: str) -> Optional[Dict[str, any]]:
    """
    Parses a text string to find and extract a function call.
    The function call is expected to be in the format:
    <functioncall> {"name": "<function_name>", "arguments": "<arguments_json_string>"}

    Args:
        input_str (str): The text containing the function call.

    Returns:
        Optional[Dict[str, any]]: A dictionary with 'name' and 'arguments' if a function call is found,
                                  otherwise None.
    """
    # Regex pattern to extract 'name' and 'arguments'
    pattern = r'"name":\s*"([^"]+)",\s*"arguments":\s*\'(.*?)\''

    # Search with regex
    match = re.search(pattern, input_str)
    if match:
        try:
            name = match.group(1)
            arguments_str = match.group(2)

            # Parse the arguments JSON
            arguments = json.loads(arguments_str)

            return {"name": name, "arguments": arguments}
        except json.JSONDecodeError:
            # If JSON parsing fails, return None
            return None
    return None


In [153]:
messages = [
     {'role': 'system','content': SYSTEM_PROMPT}, 
     {'role': 'user','content': 'What is the weather like in the Dallas, TX?'},
]

response = ollama.chat(model='natural-functions', messages=messages)
message = (response['message']['content'])

In [154]:
message

'<functioncall> {"name": "get_weather_tool", "arguments": \'{"location": "Dallas, TX"}\'}'

# Call the Function

Here we pull the function out of `kwargs` and call our tool with the arguments

In [155]:
messages.append({'role': 'assistant', 'content': message}) # add ai response to history

function_call = parse_function_call(message) # parse out function call name and args into json

if function_call and function_call.get("name") == "get_weather_tool":
    args = function_call.get("arguments")
    current_weather = get_current_weather.run(args)

current_weather

{'location': 'Dallas, TX', 'fahrenheit': 73.4}

In [156]:
messages.append({'role': 'user', 'content': 'Function Response: ' + str(current_weather)})
messages

[{'role': 'system',
  'content': '\nYou are a weather assistant with access to these functions -\n[\n    {\n        "name": "get_weather_tool",\n        "description": "get_weather_tool(location: str) -> str - Get the current weather in a given location",\n        "parameters": {\n            "type": "object",\n            "properties": {\n                "location": {\n                    "description": "The city and state, e.g. San Francisco, CA",\n                    "type": "string"\n                }\n            },\n            "required": [\n                "location"\n            ]\n        }\n    }\n]\n'},
 {'role': 'user', 'content': 'What is the weather like in the Dallas, TX?'},
 {'role': 'assistant',
  'content': '<functioncall> {"name": "get_weather_tool", "arguments": \'{"location": "Dallas, TX"}\'}'},
 {'role': 'user',
  'content': "Function Response: {'location': 'Dallas, TX', 'fahrenheit': 73.4}"}]

In [157]:
response = ollama.chat(model='natural-functions', messages=messages)
response['message']['content']

'The current weather in Dallas, TX is sunny and the temperature is around 73.4 degrees Fahrenheit'